In [28]:
:sccache 1
// Skip this or run `cargo install sccache`

sccache: true


In [29]:
:dep z3tracer
// Or use `:dep z3tracer = { path = ".." }` for the local version.

In [30]:
:dep plotters = { default_features = false, features = ["evcxr", "line_series"] }

In [31]:
:efmt {:?}

Error format: {:?} (errors must implement std::fmt::Debug)


In [32]:
:opt 3

Optimization: 3


In [33]:
use z3tracer::{syntax::*, model::*};
use std::collections::*;
use std::str::FromStr;
use plotters::prelude::*;
use plotters::evcxr::SVGWrapper;

In [34]:
:last_error_json
// BUG WORKAROUND: In case of mysteriously truncated error 'consider importing this struct', run the command above again
// to print a detailed error and find the import that is actually missing.
//
// See also :help for more useful commands

Error: 

In [35]:
/// Helper function to process files.
fn process_file(path: &str) -> std::io::Result<Model> {
    let file = std::io::BufReader::new(std::fs::File::open(path)?);
    // Inject non-default configurations here with Model::new(config).
    let mut model = Model::default();
    if let Err(le) = model.process(Some(path.to_string()), file) {
        println!("Error at {:?}: {:?}", le.position, le.error);
    }
    Ok(model)
}

/// Helper trait to print es by their id.
trait ModelExt {
    fn id2s(&self, id: &Ident) -> String;    
}

impl ModelExt for Model {
    fn id2s(&self, id: &Ident) -> String {
        self.id_to_sexp(&BTreeMap::new(), id).unwrap()
    }        
}

In [36]:
// TODO: remove after Rust issue 59278 is closed.
struct IntoIterSorted<T> {
    inner: BinaryHeap<T>,
}

impl<T> From<BinaryHeap<T>> for IntoIterSorted<T> {
    fn from(inner: BinaryHeap<T>) -> Self {
        Self { inner }
    }
}

impl<T: Ord> Iterator for IntoIterSorted<T> {
    type Item = T;

    fn next(&mut self) -> Option<T> {
        self.inner.pop()
    }

    fn size_hint(&self) -> (usize, Option<usize>) {
        let exact = self.inner.len();
        (exact, Some(exact))
    }
}

In [37]:
// Process some input file obtained with `z3 trace=true proof=true ..`
let model = process_file("../tests/data/file1.log")?;

In [38]:
// Maximum timestamp (i.e. currently, the number of lines in the Z3 log)
let max_ts = model.processed_logs();
max_ts

443166

In [39]:
// Compute top instantiated terms.
let mut top = model.most_instantiated_terms();
top.len()

70

In [40]:
// Peek at the 20th top elements
IntoIterSorted::from(top.clone()).take(20).collect::<Vec<_>>()

[(7903, basic#), (2955, datatype#85), (2955, datatype#82), (1863, datatype#), (1114, arith#), (769, datatype#59), (359, #4429!9), (359, #4328), (259, #23065!1), (232, datatype#47), (224, #20151!10), (157, #4150), (126, #23092!1), (126, #23078!2), (126, #20468!5), (123, #4347!1), (120, #4351!1), (120, #4337), (108, #23104!1), (97, datatype#53)]

In [41]:
// Compute top instantiated terms and retrieve the "timestamps" at which instantiations occur for each of the top terms.
let instantiation_times = IntoIterSorted::from(model.most_instantiated_terms()).map(|(_count, id)| {
    let mut timestamps = model.term_data(&id).unwrap().instantiations.iter().filter_map(|key| {
        let qi = model.instantiations().get(key).unwrap();
        qi.data.get(0).map(|d| model.term_data(&d.term).unwrap().timestamp)
    }).collect::<Vec<_>>();
    timestamps.sort();
    let name = match model.term(&id).unwrap() { Term::Quant { name, .. } | Term::Builtin { name: Some(name) } => name, _ => "??" };
    (name.to_string(), timestamps)
}).collect::<Vec<_>>();

In [42]:
let figure = evcxr_figure((1000, 800), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Top 10 Quantifiers Instantiations", ("Arial", 30).into_font())
        .margin(10)
        .x_label_area_size(40)
        .y_label_area_size(50)
        .build_cartesian_2d(0..max_ts, 0..max_count)?;

    chart.configure_mesh().y_desc("Cumulative number of instantiations").x_desc("Time (line number)").draw()?;

    for (j, (name, values)) in instantiation_times.iter().take(10).enumerate() {
        let color : PaletteColor<Palette9999> = PaletteColor::pick(j);
        chart
        .draw_series(
            LineSeries::new(values.iter().enumerate().map(|(i, c)| (*c, i)), &color)
        ).unwrap()
        .label(name)
        .legend(move |(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], color.filled()));
    }

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .position(SeriesLabelPosition::UpperLeft)
        .draw()?;
    Ok(())
});
figure

Top 10 Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
1000
 
 
 
2000
 
 
 
3000
 
 
 
4000
 
 
 
5000
 
 
 
6000
 
 
 
7000
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="60,749 60,748 60,748 60,748 60,748 61,748 61,748 61,748 61,748 61,748 61,748 61,748 61,747 61,747 62,747 62,747 62,747 62,747 62,747 62,747 64,747 64,747 64,747 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,746 64,745 64,745 64,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,744 65,744 67,744 67,744 68,744 68,744 68,744 68,744 68,744 68,744 68,744 70,744 70,743 70,743 70,743 81,743 82,743 82,743 82,743 82,743 82,743 85,743 85,743 85,742 85,742 85,742 85,742 85,742 86,742 86,742 86,742 86,742 86,742 86,742 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,739 86,738 86,738 86,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,737 87,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,736 88,735 89,735 89,735 89,735 89,735 89,735 92,735 92,735 92,735 92,735 92,735 92,734 92,734 92,734 94,734 94,734 94,734 94,734 94,734 95,734 95,734 95,734 95,734 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,733 95,732 95,732 95,732 95,732 95,732 96,732 96,732 96,732 96,732 96,732 96,732 96,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,729 98,729 98,729 98,729 98,729 98,729 98,729 98,729 142,729 142,729 142,729 142,729 142,728 142,728 142,728 142,728 142,728 142,728 142,728 142,728 142,728 143,728 143,728 143,727 143,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,726 144,726 144,726 144,726 144,726 145,726 145,726 145,726 145,726 145,726 145,726 145,725 145,725 145,725 145,725 145,725 145,725 145,725 146,725 146,725 146,725 146,725 146,725 146,724 146,724 146,724 146,724 146,724 146,724 146,724 146,724 146,724 147,724 147,724 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,722 147,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,721 148,721 148,721 148,721 148,721 148,721 149,721 149,721 149,721 149,721 149,721 149,720 149,720 149,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,719 150,719 150,719 151,719 151,719 151,719 151,719 151,719 151,719 151,719 151,719 151,718 151,718 151,718 151,718 151,718 151,718 151,718 151,718 151,718 152,718 152,718 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 154,715 154,714 154,714 154,714 154,714 154,714 154,714 154,714 154,714 154,714 155,714 155,714 155,713 155,713 155,713 155,713 155,713 155,713 155,713 155,713 155,713 156,713 156,713 156,712 156,712 156,712 156,712 156,712 156,712 156,712 157,712 157,712 157,712 157,712 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,710 158,710 158,710 158,710 158,710 158,710 158,710 158,710 15

In [43]:
// Top instantiated quantified term.
model.id2s(&top.peek()?.1)

"basic"

In [44]:
// Final counter and `Ident` value for the top term.
top.peek()?

(7903, basic#)

In [45]:
// Move to the first user quantifier
while !model.term(&top.peek()?.1)?.name().unwrap_or("").starts_with("outputbpl") { top.pop(); }
top.peek()?

(359, #4429!9)

In [46]:
// Introspect a particular term, given by handle.
// Note that the version number `!9` was added during parsing for disambiguation and is not present in the original log file.
model.term(&Ident::from_str("#4429!9")?)?

Quant { name: "outputbpl.310:31", params: 2, triggers: [#284], body: #4443, var_names: Some([VarName { name: Symbol("v2"), sort: Symbol("T@$Value") }, VarName { name: Symbol("v1"), sort: Symbol("T@$Value") }]) }

In [47]:
model.term_data(&Ident::from_str("#4429!9")?)?.instantiations.len()

359

In [48]:
// Create a new chart only with quantifier instantiation inside the file `outputbpl`.
let user_instantiation_times = instantiation_times.clone().into_iter().filter(|(n, _)| n.starts_with("outputbpl")).collect::<Vec<_>>();

let max_user_count = user_instantiation_times[0].1.len();

let figure = evcxr_figure((1000, 800), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Top 10 User Quantifiers Instantiations", ("Arial", 30).into_font())
        .margin(10)
        .x_label_area_size(40)
        .y_label_area_size(50)
        .build_cartesian_2d(0..max_ts, 0..max_user_count)?;

    chart.configure_mesh().y_desc("Cumulative number of instantiations").x_desc("Time (line number)").draw()?;

    for (j, (name, values)) in user_instantiation_times.iter().take(10).enumerate() {
        let color : PaletteColor<Palette9999> = PaletteColor::pick(j);
        chart
        .draw_series(
            LineSeries::new(values.iter().enumerate().map(|(i, c)| (*c, i)), &color)
        ).unwrap()
        .label(name)
        .legend(move |(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], color.filled()));
    }

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .position(SeriesLabelPosition::UpperLeft)
        .draw()?;
    Ok(())
});
figure

Top 10 User Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="255,749 256,746 256,744 290,742 292,740 293,738 296,736 296,734 297,731 301,729 302,727 305,725 307,723 308,721 308,719 309,717 341,714 341,712 342,710 346,708 347,706 347,704 348,702 351,700 352,697 386,695 386,693 387,691 391,689 393,687 394,685 394,682 395,680 396,678 397,676 398,674 398,672 399,670 400,668 400,665 400,663 402,661 403,659 403,657 404,655 404,653 404,651 405,648 405,646 406,644 447,642 448,640 448,638 449,636 449,633 450,631 450,629 451,627 451,625 452,623 452,621 452,619 452,616 452,614 452,612 452,610 452,608 452,606 452,604 452,602 452,599 452,597 452,595 453,593 454,591 454,589 455,587 456,584 456,582 456,580 456,578 456,576 456,574 456,572 456,570 456,567 456,565 456,563 456,561 457,559 457,557 457,555 458,553 459,550 460,548 460,546 461,544 461,542 462,540 480,538 481,536 481,533 481,531 482,529 482,527 483,525 483,523 483,521 483,518 483,516 483,514 483,512 483,510 489,508 490,506 491,504 511,501 512,499 513,497 514,495 516,493 517,491 533,489 534,487 535,484 535,482 536,480 547,478 547,476 551,474 551,472 553,469 554,467 556,465 557,463 557,461 557,459 557,457 569,455 573,452 574,450 574,448 583,446 584,444 584,442 585,440 585,438 586,435 586,433 587,431 587,429 593,427 594,425 612,423 612,420 615,418 617,416 617,414 618,412 619,410 620,408 620,406 624,403 625,401 626,399 626,397 631,395 632,393 632,391 633,389 634,386 635,384 638,382 638,380 652,378 652,376 654,374 655,372 656,369 656,367 657,365 657,363 657,361 657,359 657,357 657,354 657,352 658,350 658,348 659,346 659,344 660,342 660,340 661,337 661,335 663,333 664,331 664,329 665,327 665,325 666,323 666,320 667,318 667,316 668,314 668,312 669,310 669,308 669,305 686,303 686,301 687,299 687,297 687,295 687,293 687,291 687,288 687,286 687,284 688,282 688,280 688,278 688,276 688,274 688,271 688,269 688,267 689,265 693,263 694,261 694,259 694,256 694,254 695,252 712,250 712,248 714,246 715,244 715,242 715,239 719,237 722,235 723,233 723,231 724,229 725,227 725,225 726,222 728,220 728,218 728,216 729,214 740,212 741,210 743,208 743,205 744,203 744,201 744,199 745,197 760,195 760,193 762,190 762,188 762,186 763,184 772,182 772,180 779,178 779,176 780,173 780,171 788,169 789,167 795,165 795,163 795,161 796,159 797,156 797,154 799,152 800,150 800,148 801,146 819,144 819,141 819,139 820,137 821,135 822,133 842,131 842,129 842,127 847,124 848,122 849,120 849,118 850,116 870,114 871,112 871,110 871,107 871,105 872,103 873,101 893,99 898,97 901,95 902,92 902,90 921,88 921,86 921,84 922,82 922,80 924,78 925,75 925,73 942,71 949,69 950,67 950,65 951,63 967,61 969,58 970,56 970,54 970,52 971,50 983,48 984,46 "/>
<polyline fill="none" opacity="1" stroke="#0082C8" stroke-width="1" points="255,749 255,746 256,744 290,742 291,740 292,738 295,736 296,734 296,731 301,729 305,727 307,725 307,723 308,721 308,719 340,717 341,714 341,712 346,710 346,708 347,706 347,704 350,702 351,700 351,697 385,695 386,693 387,691 389,689 391,687 393,685 393,682 394,680 394,678 395,676 397,674 397,672 398,670 398,668 399,665 399,663 400,661 400,659 400,657 402,655 403,653 403,651 403,648 404,646 404,644 406,642 447,640 448,638 448,636 448,633 449,631 450,629 451,627 451,625 451,623 452,621 452,619 452,616 452,614 452,612 452,610 452,608 452,606 452,604 452,602 452,599 452,597 452,595 453,593 